In [1]:
%matplotlib inline
from ipywidgets import interactive, fixed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
from scipy import stats

## Beta-distribution

Beta-distribution 
$$ \beta(x, a, b) = c x^{a-1}(1-x)^{b-1}, $$ where $c$ is the normalization factor,
$$c = \frac{\Gamma(a+b)}{\Gamma(a)\Gamma(b)}.$$

In Bayesian probability theory's terms, the beta-distribution and the binomial distributions are conjugate distribution; meaning if the prior takes the form of the beta-distribution, the likelihood the binomial, then the posterior distribution will be a beta-distribution too. 

Indeed, given a beta-distribution prior $$f(p)=\beta(p; a_0, b_0)$$ and likelihood $$L(a, b|p)=\text{binom}(a, b; p)=C_{a+b}^a p^a(1-p)^b,$$ multiply the two and we have a posterior of $p$, $$P(p)=d\cdot p^{a+a_0-1}(1-p)^{b+b_0-1}=\beta(a+a_0, b+b_0).$$ where $d$ is a normalization factor. 

This gives a simple update-rule, i.e., the probability of an event is given by $\beta(x; a, b)$, where 
$$a \leftarrow a_{obs} + a_{0},$$
$$b \leftarrow b_{obs} + b_{0}. $$

In [3]:
def beta_pdf(a=1, b=1):
    x = np.linspace(0, 1, 100)
    y = stats.beta.pdf(x, a, b)
    plt.figure(2)
    plt.plot(x, y)
    plt.ylim(0, max(y)+1)
    plt.show()

In [4]:
interactive_plot = interactive(beta_pdf, a=(1, 5), b=(1, 30))
output = interactive_plot.children[-1]
# output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=1, description='a', max=5, min=1), IntSlider(value=1, description='b', m…

## Read data & Preprocess

In [5]:
data_file = "data/Chapter-9-Beta-Dist-Example-for-three-industries-1.xlsx"

In [6]:
header = pd.read_excel(io=data_file, nrows=1, usecols=[0])

In [7]:
from IPython.display import Markdown as md
md("%s" % (header.iloc[0][0]
           .replace("“alpha”", "**“a”**")
           .replace("“beta”", "**“b”**")))

This shows how the beta distribution could be used to compare breach frequencies based on a few breaches in an industry.  Data from 2014 to the end of 2015 is shown.  You can set **“a”** and **“b”** as shown in the book to reflect “hits” and “misses” (i.e., breaches and non-breaches per company per year) to see how the estimate of breach frequency will change with even a single new breach reported.  In rows 4 to 7 (in yellow) you can enter the start year of the breach data for an industry, the end year, the number of breaches in that period, and the number of organizations in the sample.  These assume the organizations are either randomly sampled from some industry or a complete census of the industry.  Data shown was based on a particular random sample and was accurate until the beginning of 2016.

In [8]:
data = pd.read_excel(io=data_file, nrows=4, skiprows=2, usecols=[0, 1, 2, 3], index_col=0)

In [9]:
data = data.T

In [10]:
data['duration'] = data['Data up until year'] - data['Data since year']

In [11]:
data

,Data since year,Data up until year,Breaches since then,Companies in Population,duration
Healthcare,2014,2016,2,38,2
Retail,2014,2016,3,98,2
Finance,2014,2016,2,60,2


## Healthcare as an example

In [12]:
def beta_posterior_plot(prior_a=1, prior_b=1, hit=0, miss=0):
    plt.figure(2)
    x = np.linspace(0, 1, 100)

    post_a = prior_a+hit
    post_b = prior_b+miss

    y = stats.beta.pdf(x, post_a, post_b)

    mean = post_a/(post_a+post_b)  # occurrence rate
    lower, upper = stats.beta.interval(0.90, post_a, post_b)
    plt.plot(x, y)
    plt.ylim(0, max(y)+1)
    plt.title("(prior) a=%d, b=%d\n (observed) hit=%d, miss=%d\n" % (prior_a, prior_b, hit, miss) +\
              "Mean=%.4f\n 90%% CI lower=%.2f%%, upper=%.2f%%" % (mean, lower*100, upper*100) )
    plt.show()

In [13]:
hc = data.loc['Healthcare']

In [14]:
hc

Data since year            2014
Data up until year         2016
Breaches since then           2
Companies in Population      38
duration                      2
Name: Healthcare, dtype: int64

In [15]:
hc_hit = hc['Breaches since then']
hc_miss = hc['Companies in Population'] * hc['duration'] - hc_hit
interactive_plot = interactive(beta_posterior_plot, a=(1, 5), b=(1, 30), 
                               hit=fixed(hc_hit), 
                               miss=fixed(hc_miss))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=1, description='prior_a', max=3, min=-1), IntSlider(value=1, description…

Note although we observed 2 breaches and 74 misses, the data breach probability is *not* 2/76=2.63% -- the observed counts are random variables, hence their ratio is a random variable. Applying the beta-distribution (and assuming a uniform prior), we know there is a non-negaligible (5%) possibility that the true probability reaches 8%. 